In [1]:
#### ~~~ modules of Python standard libraries ~~~ ####
import pandas as pd 
import numpy as np 
import sys 
import os
import re
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
#### ~~~ user-defined modules ~~~~ ####
sys.path.append("/Users/gongyun/ResearchPlatform/ResearchTool/Simulation/SimPy3/")
# ~ instrument session ~ #
from config import session_spans_sc_ as session_spans
from library.prepareData import prepareData
from library.simulator.StatsCompare import StatsCompare
from library.simParams import simParamsInstance
from strategyTemplate.strategyParams import paramsInstance
sys.path.append("/Users/gongyun/ResearchPlatform/ResearchTool/")
from ResearchUtil import *

In [2]:
### ~~~ configure ~~~ ###
instrument = "OI"
volumeRank = 1
tradeInstrument = instrument.upper()+"000"+str(volumeRank)
mainAssistInstrument = "P0001"
startDate = "20210125"
endDate = "20210125"
feeReturnRate = 0.5
FEE = 1 *(1-feeReturnRate)
openRate = 0.0*(1-feeReturnRate)
closeRate = 0.0*(1-feeReturnRate)
useRealPredictor = False
useOrderLog = True
strategy = "xbow"
simResultName = strategy

priceTick = 1
multiplier = 10

sim_data_dir = '/Users/gongyun/ResearchPlatform/ResearchData/SimulationData/MarketData/'
sim_order_dir = '/Users/gongyun/ResearchPlatform/ResearchData/SimulationData/OrderData/'
sim_result_dir = '/Users/gongyun/ResearchPlatform/ResearchData/SimulationData/ResultData/'

### ~~~ get mkt dates from sim data ~~~ ###     
l_date = []    
for dirpath,dirnames,filenames in os.walk(sim_data_dir):
    for filename in filenames:
        if ("mkt_data" in filename) and (".h5" not in filename):
            date = filename[-12:-4]
            if (date >= startDate) and (date <= endDate): 
                l_date.append(date)  
l_date.sort()
### ~~~ check order log ~~~ ###
d_useOrderLog = {}
if useOrderLog == False:
    for date in l_date:
        d_useOrderLog[date]=False
elif useOrderLog == True:
    for date in l_date:
        if os.path.exists(sim_order_dir+tradeInstrument+"_order_data_"+date+".csv"):
            d_useOrderLog[date]=True
        else:
            d_useOrderLog[date]=False
            print("There is no order log on "+date)

In [3]:
def simulation(date,tradeInstrument,mainAssistInstrument,strategy,sim_order_dir,useOrderLog,sim_data_dir,sim_result_dir):
    ### ~~~ prepare data ~~~ ###
    useOrderLog = d_useOrderLog[date]
    if useOrderLog:
        data = prepareData(
	        sim_datafile_full_path = sim_data_dir+tradeInstrument+"_mkt_data_"+date+".csv",
	        session_spans = session_spans,  
	        doubleVolumeFlag = False,
            order_log_full_path = sim_order_dir+tradeInstrument+"_order_data_"+date+".csv",
	        )
    else:
        data = prepareData(
	        sim_datafile_full_path = sim_data_dir+tradeInstrument+"_mkt_data_"+date+".csv",
	        session_spans = session_spans,  
	        doubleVolumeFlag = False,
	        )        
    print("finish")
    
    insRootDict = {tradeInstrument:0,'P0001':1,'Y0001':2,'RM0001':3,}    
    
    #### simulation
    stats = StatsCompare(
	    dataInputGen = data,
	    paramsList = [ 
		    [0.4], [0.4], [0.4],        #'fracCut','fractCutImprove', 'fractCutTake',
            [2], [100],[1],             #'maxUnitOfRisk','oneSidedMaxPos','maxQuoteSize',
            [1.0], [0.3], [0.5], [0.1], #'posPenIntercept', 'posPenCoef', 'posPenLowerBound','posPenLowerBoundCancel',
            [0.1], [0.0],               #'minimumEdge', 'minimumEdgeRate',
            [1.0],[0.05],[10],         #'alpha','signalScoreB','signalScoreC',
            [0.0], [0],                 #'signalFloor','qI',
            [0.25],[0.99], [1000.0], [1000.0],[-1000], #'initVar', 'varEwmaDecay', 'volBound4Joining', 'volBound4Improving','volBound4Taking',
            [4],                               #'quoteRange',            
            [1.0],                             #'improveTick',  
            [0.6],[0.1],[0.1],[0.3],           #'theoGamma','theoAlpha','theoGamma2','theoAlpha2',
            [-1.0],[-1.0],                     #'minSnapshotVolumeTake', 'minStandizedSignalForTaking',
            [False],[0.3], [1.0], [0.2],        #'useExt','minEdge2','cmeScaler','cancelScalerOnAssist',
            #[3],[-1],[-1],[-1],[-1],            #'notActivatedAssistInsRoot_1','notActivatedAssistInsRoot_2','notActivatedAssistInsRoot_3','notActivatedAssistInsRoot_4','notActivatedAssistInsRoot_5',    
            #[-1],[1],[2],                    #'useTheoReturnAssistInsRoot_1','useTheoReturnAssistInsRoot_2','useTheoReturnAssistInsRoot_3',        
            #[True],[0.3],[0.3],[0.3],[0.3],    #'useAssistReturn','assistReturnBeta_1','assistReturnBeta_2','assistReturnBeta_3_H','assistReturnBeta_3_T',
            [0.1],[1.0],[0.0],                #'varAlpha4T','varBeta4T', 'varBase4T',
            [0.2],[1.0],[0.0],                #'varAlpha4I','varBeta4I', 'varBase4I',
            [0.4],[1.0],[0.0],                #'varAlpha4J','varBeta4J', 'varBase4J',
            [3000],
            [False],[0.0],
            [False],[0.0],
            [False],[-40],
            [1.0],                            #'queueEdgeAlpha',
            #[0.0],[0.0],[0.0],               #'G_Factor1','G_Factor2','G_Factor3',
            #[0.0],[0.0],[0.0],               #'Y_Factor1','Y_Factor2','Y_Factor3',
            #[0.8],
            ],
        paramNames = [ 
            'fracCut','fractCutImprove', 'fractCutTake',
            'maxUnitOfRisk','oneSidedMaxPos','maxQuoteSize',
            'posPenIntercept', 'posPenCoef', 'posPenLowerBound','posPenLowerBoundCancel',
            'minimumEdge', 'minimumEdgeRate',
            'alpha','signalScoreB','signalScoreC',
            'signalFloor','qI',
            'initVar', 'varEwmaDecay', 'volBound4Joining', 'volBound4Improving','volBound4Taking',
            'quoteRange',   
            'improveTick',     
            'theoGamma','theoAlpha','theoGamma2','theoAlpha2',
            'minSnapshotVolumeTake', 'minStandizedSignalForTaking',
            'useExt', 'minEdge2', 'cmeScaler','cancelScalerOnAssist',
            #'notActivatedAssistInsRoot_1','notActivatedAssistInsRoot_2','notActivatedAssistInsRoot_3','notActivatedAssistInsRoot_4','notActivatedAssistInsRoot_5',  
            #'useTheoReturnAssistInsRoot_1','useTheoReturnAssistInsRoot_2','useTheoReturnAssistInsRoot_3',        
            #'useAssistReturn','assistReturnBeta_1','assistReturnBeta_2','assistReturnBeta_3_H','assistReturnBeta_3_T',
            'varAlpha4T','varBeta4T', 'varBase4T',
            'varAlpha4I','varBeta4I', 'varBase4I',
            'varAlpha4J','varBeta4J', 'varBase4J',
            'spoofingLevel',
            'useIncreAssistTheo','increAssistTheoThreshold',
            'useIncreAssistSignal','increAssistSignalThreshold',
            'useNewCancel','probCancel',
            'queueEdgeAlpha',
            #'G_Factor1','G_Factor2','G_Factor3',
            #'Y_Factor1','Y_Factor2','Y_Factor3',
            #'L_Factor1',
            ],
        simParamsList = [
            [False],[False],[useOrderLog],[FEE/multiplier],[openRate],[closeRate],[np.timedelta64(100000000, 'ns')]#[10.0/1000]
            ],
        simParamNames = [
            'fill1MktVolumeFilter','fill1VolumeConstraintFilter','useOrderLog','PERROUNDFEE','openRate','closeRate','orderLagTime',
            ],
        )

    stats.crossCompare(
        params = paramsInstance,
        simParams = simParamsInstance,
        tradeInsRoot = tradeInstrument,
        signalName = 'Pred',
        insRootDict = insRootDict,
        strategyModule = strategy,   
        orderManagerModule = 'orderManager',
        dataLenCut=10
        )

    stats.getStats(cancelFailWeight=1.1498,takingFailWeight=0.2482,improvingFailWeight=0.2498,joinFailWeight=0.2244,allFailWeight=0.0)
    stats.getTradeStats()
    
    #### ~~~ simulation results ~~~ ####
    df_table = pd.DataFrame(
    np.array([
             (stats.statsResults.allSuccessTerminal / stats.statsResults.allSuccessTradeNum).values
            ,(stats.statsResults.cancelFailedTerminal / stats.statsResults.cancelFailedTradeNum).values
            ,(stats.statsResults.joinFailedTerminal / stats.statsResults.joinFailedTradeNum).values
            ,(stats.statsResults.takingFailedTerminal / stats.statsResults.takingFailedTradeNum).values
            ,(stats.statsResults.improvingFailedTerminal / stats.statsResults.improvingFailedTradeNum).values
            ,(stats.statsResults.allFailedTerminal / stats.statsResults.allFailedTradeNum).values
            ,(stats.statsResults.terminal / stats.statsResults.allSuccessTradeNum).values
        ]).T * multiplier
        , columns=['AllSuccess','CancelF','JoinF', 'TakeF','ImproveF', 'AllF', 'Average']
    )

    df_parameter = stats.statsResults.T
    
    ### ~~~ save sim results ~~~ ###
    if not os.path.exists(sim_result_dir+date+"/"):
        os.makedirs(sim_result_dir+date+"/")

    df_table.to_csv(sim_result_dir+date+"/"+instrument+"_table_"+date+".csv",index=False)
    df_parameter.to_csv(sim_result_dir+date+"/"+instrument+"_parameter_"+date+".csv",index=True)
    ### ~~~ save sim order results ~~~ ###
    for i in range(6):
        pd.DataFrame(stats.tradesFrameListOfListByParams[0][0][i]).to_csv(sim_result_dir+date+"/"+instrument+"_trade_log_"+date+"_s"+str(i)+".csv",index=False)
    for j in range(6):
        pd.DataFrame(stats.ordersFrameListOfListByParams[0][0][j]).to_csv(sim_result_dir+date+"/"+instrument+"_order_log_"+date+"_s"+str(j)+".csv",index=False)
    print("#######################################")
    print("####### "+date+" sim results ##########")
    print(df_table)
    print(df_parameter)
    print("#######################################")     

In [4]:
for date in l_date:
    simulation(date,tradeInstrument,mainAssistInstrument,strategy,sim_order_dir,useOrderLog,sim_data_dir,sim_result_dir)
print("Simulation Complete")

Preparing market data...
Preparing order log data...
Combining orders...


0it [00:00, ?it/s]


finish
Trading instrument: 'OI0001'
Using strategy module: 'kronos_base'
Using orderManager module: 'orderManager'
Parameter set 1 out of 1:
Using order log: True
Scenario 1 out of 6:


100%|██████████████████████████████████| 1/1 [00:17<00:00, 17.45s/it]


Scenario 2 out of 6:


...

Scenario 5 out of 6:


100%|██████████████████████████████████| 1/1 [00:00<00:00, 12.19it/s]


Scenario 6 out of 6:


100%|██████████████████████████████████| 1/1 [00:00<00:00, 12.24it/s]


#######################################
####### 20210125 sim results ##########
   AllSuccess  CancelF     JoinF     TakeF  ImproveF      AllF   Average
0    3.791237  3.72762  3.771967  3.768692  3.789874  3.649371  3.729712
                                           0
sharpe                                   NaN
terminal                           723.56411
sessionlyDrawDown                        0.0
sessionlyWinningProb                     1.0
maxDailyLoss                       723.56411
...                                      ...
useOrderLog                             True
PERROUNDFEE                             0.05
openRate                                 0.0
closeRate                                0.0
orderLagTime          0 days 00:00:00.100000

[86 rows x 1 columns]
#######################################
Simulation Complete
